In [377]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from bs4 import BeautifulSoup

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]



In [378]:
lps = ([lp for cp in get_category_pages('/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D8%A2%D8%AE%D8%B1,آخر
1,/wiki/%D8%A2%D8%B3%D9%81,آسف
2,/wiki/%D8%A2%D9%84%D8%A7%D9%81,آلاف
3,/wiki/%D8%A2%D9%84%D8%A9,آلة
4,/wiki/%D8%A3%D8%A8%D8%A7%D8%AC%D9%88%D8%B1,أباجور
...,...,...
478,/wiki/%D9%8A%D8%A7,يا
479,/wiki/%D9%8A%D8%A7_%D8%B1%D9%8A%D8%AA,يا ريت
480,/wiki/%D9%8A%D9%84%D8%A7,يلا
481,/wiki/%D9%8A%D9%85%D9%8A%D9%86,يمين


In [390]:
def get_random_soup():
    idx = random.randint(0, len(lemmas)-1)
    row = lemmas.iloc[idx]
    print(row.title)
    return get_soup(row.href)
    
random_soup = get_random_soup()

get_headwords(random_soup)[0]

ارتاح


In [397]:
print(get_headwords(random_soup)[0].prettify())

<p>
 <strong class="Arab headword" lang="ajp">
  ارتاح
 </strong>
 <a class="mw-redirect" href="/wiki/Wiktionary:South_Levantine_Arabic_transliteration" title="Wiktionary:South Levantine Arabic transliteration">
  •
 </a>
 (
 <span class="headword-tr manual-tr tr Latn" dir="ltr" lang="ajp-Latn">
  irtāḥ
 </span>
 ) (
 <i>
  <a href="/wiki/Appendix:Arabic_verbs#Form_VIII" title="Appendix:Arabic verbs">
   form VIII
  </a>
 </i>
 ,
 <i>
  present
 </i>
 <b class="Arab" lang="ajp">
  <a class="new" href="/w/index.php?title=%D8%A8%D8%B1%D8%AA%D8%A7%D8%AD&amp;action=edit&amp;redlink=1" title="برتاح (page does not exist)">
   برتاح
  </a>
 </b>
 ‎
 <span class="mention-gloss-paren annotation-paren">
  (
 </span>
 <span class="tr Latn" lang="ajp-Latn">
  birtāḥ
 </span>
 <span class="mention-gloss-paren annotation-paren">
  )
 </span>
 ,
 <i>
  active participle
 </i>
 <b class="Arab" lang="ajp">
  <a href="/wiki/%D9%85%D8%B1%D8%AA%D8%A7%D8%AD#South_Levantine_Arabic" title="مرتاح">
   مرتاح
 

In [411]:
get_headwords(random_soup)[0]

'Verb'

In [412]:
def get_headwords(soup, language="Levantine_Arabic"):
    return soup.select(f'div#bodyContent h2:has(span[id$="{language}"]) ~ p:has(.headword)')

def get_definitions(headword):
    lemma = headword.strong.get_text()
    headline = headword.find_previous(class_='mw-headline').string
    print(lemma)

    def_ol = headword.find_next_sibling('ol')
    
    results = []
    for li in def_ol.select('li'):
        result = {'lemma': lemma, 'headline': headline}
        definition = ''
        for s in li.strings:
            if 'dl' in list(p.name for p in s.parents):
                break
            definition += s
        result['definition'] = definition.strip()
        if li.dl:
            detail = '\n'.join(e.get_text() for e in  li.dl.find_all(class_=re.compile('^e')))
            result['detail'] = detail.strip()
        results.append(result)
    return results


In [413]:
get_definitions(get_headwords(random_soup)[0])

ارتاح


[{'lemma': 'ارتاح', 'headline': 'Verb', 'definition': 'to rest, to relax'},
 {'lemma': 'ارتاح',
  'headline': 'Verb',
  'definition': 'to be at ease or comfortable with something',
  'detail': "انا بالمرّة ما ارتحت.\nana bil-marra mā rtaḥt\nI wasn't comfortable at all.\nخبّرني إذا فيه أي إشي ما ارتحتيله.\nḵabbirni ʔiza fīh ʔayy ʔiši mā rtaḥtīlo\nLet me know if there's anything you're not comfortable with."}]

In [414]:
Definitions = pd.DataFrame([definition for path in lemmas.href.values for hw in get_headwords(get_soup(path)) for definition in get_definitions(hw)])

آخر
آخر
آخر
آخر
آسف
آلاف
آلة
أباجور
أبعد
أبيض
أثّر
أجار
أجّل
أجنبي
أجنبي
أحمر
أخد
أخضر
أدنى
أرجل
أرجيلة
أرض
أرمني
أرمني
أرنب
أزرق
أزمة
أساس
أسبوع
أستاذ
أسد
أسفل
أسفل
أسود
أشّر
أشهر
أصبع
أصفر
أضفر
أعلى
أغلب
أفكار
أقصى
أكتر
أكتر
أكّد
أَكْزَخَانة
أكل
أكل
أكم
ألهى
أم
أما
أمّن
أنعم
أنعم
أهان
أو
أوضة
أوضة
أول
أول
أيّام
إجا
إجا
إجا عالوجع
إجا يكحّلها عماها
إذا
إذا
إذن
إسلام
إشي
إشي
إلـ
إلّا
إمّا
إن
إنّـ
إنكنّـ
احتاج
احتياط
احتياط
احمرّ
اختار
اخترع
اختفى
ارتاح
استأجر
استحى
استخدم
استراحة
استرخى
استشار
استعار
استعمل
استغرب
استغل
استغنى
استفاد
استقبل
استنّى
استورد
اسم
اسم
اِسْم
اسم
اسمر
اشترك
اشترى
اشتغل
اعتبر
افتخر
الـ
ال
ال
ال
الـ
التقى
الحمد لله
اللي
اللي
امتنع
انباع
انتخب
انتقل
انتمى
انحبّ
اندمج
انفتح
انفكّ
انقلع
انكبّ
اهتمّ
ايّا
ايا
ايد
إيد
بارد
بارك
باس
باس
باس
باص
باع
بال
بال
بال
بال
بالمرة
بالمية
باي
بخبخ
بد
بد
بد
بد
بَد
بَد
بد
بدّ
بد
بد
بدا
بدّع
بدّل
بدل
برد
برّد
برد
برد
بردقان
برقوق
بسّ
بسّ
بسّ
بسّ
بَس
بَس
بس
بس
بَس
بس
بس
بس
بس
بس
بطاطا
بطاقة
بطّل
بعت
بعد
بعد
بعد
بعد
بعد
بعد
بعد
بعد
بع

In [415]:
Definitions

,lemma,headline,definition,detail
0,آخر,Noun,end,آخر الأسبوع\nʔāḵer il-ʔusbūʕ\nthe weekend
1,آخر,Adjective,"last, final",هاي آخر مرّة بستنّاك.\nhāy ʔāḵer marra bastann...
2,آخر,Adjective,"last, most recent",مش عارف إيمتى آخر مرّة سمعت هالجملة.\nmiš ʕāre...
3,آخر,Adverb,after all,NaN
4,آخر,Adverb,at last,NaN
...,...,...,...,...
1257,يلّا,Interjection,"let's, let's go",يلا نروح على بركة.\nyaḷḷa nrūḥ ʕala birke\nLet...
1258,يلّا,Interjection,"(by extension) bye-bye, bye",يلا باي\nyaḷḷa ḅāy\nbye-bye
1259,يمين,Noun,right (side or direction),لف عإيدك اليمين!\nliff ʕa-ʔīdak il-yamīn!\nTur...
1260,ـين,Suffix,Dual suffix,


In [418]:
Definitions[Definitions.headline == 'Inflection']

,lemma,headline,definition,detail
858,في,Inflection,"there is, there are",شو في؟\nšū fī?\nWhat's up?\nفي إشي؟\nfī ʔiši?\...


In [419]:
Definitions.set_index('lemma').join(lemmas.set_index('title')).reset_index().to_csv('data/South_Levantine_Arabic_terms_with_usage_examples.csv')

In [386]:
pd.DataFrame([definition for soup in soups.values() for hw in get_headwords(soup) for definition in get_definitions(hw)])


آخر
آخر
آخر
آخر
آسف
آلاف
آلة
أباجور
أبعد
أبيض
أثّر
أجار
أجّل
أجنبي
أجنبي
أحمر
أخد
أخضر
أدنى
أرجل
أرجيلة
أرض
أرمني
أرمني
أرنب
أزرق
أزمة
أساس
أسبوع
أستاذ
أسد
أسفل
أسفل
أسود
أشّر
أشهر
أصبع
أصفر
أضفر
أعلى
أغلب
أفكار
أقصى
أكتر
أكتر
أكّد
أَكْزَخَانة
أكل
أكل
أكم
ألهى
أم
أما
أمّن
أنعم
أنعم
أهان
أو
أوضة
أوضة
أول
أول
أيّام
إجا
إجا
إجا عالوجع
إجا يكحّلها عماها
إذا
إذا
إذن
إسلام
إشي
إشي
إلـ
إلّا
إمّا
إن
إنّـ
إنكنّـ
احتاج
احتياط
احتياط
احمرّ
اختار
اخترع
اختفى
ارتاح
استأجر
استحى
استخدم
استراحة
استرخى
استشار
استعار
استعمل
استغرب
استغل
استغنى
استفاد
استقبل
استنّى
استورد
اسم
اسم
اِسْم
اسم
اسمر
اشترك
اشترى
اشتغل
اعتبر
افتخر
الـ
ال
ال
ال
الـ
التقى
الحمد لله
اللي
اللي
امتنع
انباع
انتخب
انتقل
انتمى
انحبّ
اندمج
انفتح
انفكّ
انقلع
انكبّ
اهتمّ
ايّا
ايا
ايد
إيد
بارد
بارك
باس
باس
باس
باص
باع
بال
بال
بال
بال
بالمرة
بالمية
باي
بخبخ
بد
بد
بد
بد
بَد
بَد
بد
بدّ
بد
بد
بدا
بدّع
بدّل
بدل
برد
برّد
برد
برد
بردقان
برقوق
بسّ
بسّ
بسّ
بسّ
بَس
بَس
بس
بس
بَس
بس
بس
بس
بس
بس
بطاطا
بطاقة
بطّل
بعت
بعد
بعد
بعد
بعد
بعد
بعد
بعد
بعد
بع

,lemma,definition,detail
0,آخر,end,آخر الأسبوع\nʔāḵer il-ʔusbūʕ\nthe weekend
1,آخر,"last, final",هاي آخر مرّة بستنّاك.\nhāy ʔāḵer marra bastann...
2,آخر,"last, most recent",مش عارف إيمتى آخر مرّة سمعت هالجملة.\nmiš ʕāre...
3,آخر,after all,NaN
4,آخر,at last,NaN
...,...,...,...
1257,يلّا,"let's, let's go",يلا نروح على بركة.\nyaḷḷa nrūḥ ʕala birke\nLet...
1258,يلّا,"(by extension) bye-bye, bye",يلا باي\nyaḷḷa ḅāy\nbye-bye
1259,يمين,right (side or direction),لف عإيدك اليمين!\nliff ʕa-ʔīdak il-yamīn!\nTur...
1260,ـين,Dual suffix,


In [289]:
hws = get_headwords(list(soups.values())[0])

get_definitions(hws[0]) 

p
بردقان
<p><strong class="Arab headword" lang="ajp">بردقان</strong> <a class="mw-redirect" href="/wiki/Wiktionary:South_Levantine_Arabic_transliteration" title="Wiktionary:South Levantine Arabic transliteration">•</a> (<span class="headword-tr manual-tr tr Latn" dir="ltr" lang="ajp-Latn">burdʔān</span>) <span class="gender"><abbr title="masculine gender">m</abbr></span> (<i>collective</i>, <i>singulative</i> <b class="Arab" lang="ajp"><a class="new" href="/w/index.php?title=%D8%A8%D8%B1%D8%AF%D9%82%D8%A7%D9%86%D8%A9&amp;action=edit&amp;redlink=1" title="بردقانة (page does not exist)">بردقانة</a></b>‎ <span class="gender"><abbr title="feminine gender">f</abbr></span> <span class="mention-gloss-paren annotation-paren">(</span><span class="tr Latn" lang="ajp-Latn">burdʔāne</span><span class="mention-gloss-paren annotation-paren">)</span>, <i>plural</i> <b class="Arab" lang="ajp"><a class="new" href="/w/index.php?title=%D8%A8%D8%B1%D8%AF%D9%82%D8%A7%D9%86%D8%A7%D8%AA&amp;action=edit&amp;r

[{'lemma': 'بردقان',
  'definition': '(uncountable) oranges (fruits)',
  'detail': 'حبة بردقان / بردقانة\nḥabbet burdʔān / burdʔāne\nan orange'}]

In [270]:
get_headwords(soup)

[<p><strong class="Arab headword" lang="ajp">انحبّ</strong> <a class="mw-redirect" href="/wiki/Wiktionary:South_Levantine_Arabic_transliteration" title="Wiktionary:South Levantine Arabic transliteration">•</a> (<span class="headword-tr manual-tr tr Latn" dir="ltr" lang="ajp-Latn">inḥabb</span>) (<i><a href="/wiki/Appendix:Arabic_verbs#Form_VII" title="Appendix:Arabic verbs">form VII</a></i>, <i>present</i> <b class="Arab" lang="ajp"><a class="new" href="/w/index.php?title=%D8%A8%D9%86%D8%AD%D8%A8&amp;action=edit&amp;redlink=1" title="بنحب (page does not exist)">بنحبّ</a></b>‎ <span class="mention-gloss-paren annotation-paren">(</span><span class="tr Latn" lang="ajp-Latn">binḥabb</span><span class="mention-gloss-paren annotation-paren">)</span>)
 </p>]

In [243]:
hw[0].strong.string

'انحبّ'

In [164]:
headword = soup.select_one('div#bodyContent h2:has(span[id$="Levantine_Arabic"]) ~ p:has(.headword)')
header = headword.find_previous_sibling(['h3', 'h4'])
header_name = header.name

def_ol = headword.find_next_sibling('ol')

<ol>
 <li>
  <span class="ib-brac qualifier-brac">
   (
  </span>
  <span class="ib-content qualifier-content">
   uncountable
  </span>
  <span class="ib-brac qualifier-brac">
   )
  </span>
  <a href="/wiki/eggs" title="eggs">
   eggs
  </a>
 </li>
</ol>



In [149]:
import yaml

print(yaml.dump([{'href': '/wiki/l%E1%BB%8D_m%E1%BB%8D', 'title': 'lọ mọ'},
 {'href': '/wiki/lo%E1%BA%A1n_x%E1%BA%A1', 'title': 'loạn xạ'},
 {'href': '/wiki/l%E1%BB%8Fm', 'title': 'lỏm'}], allow_unicode=True))

- href: /wiki/l%E1%BB%8D_m%E1%BB%8D
  title: lọ mọ
- href: /wiki/lo%E1%BA%A1n_x%E1%BA%A1
  title: loạn xạ
- href: /wiki/l%E1%BB%8Fm
  title: lỏm



In [29]:
query = urlparse('/w/index.php?title=Category:Vietnamese_adverbs&pagefrom=D%EF%80%80ANG+KHO%EF%80%80NG%0A%C4%91ang+kh%C3%B4ng#mw-pages').query
quote(parse_qs(query)['pagefrom'][0])

'D%EF%80%80ANG%20KHO%EF%80%80NG%0A%C4%91ang%20kh%C3%B4ng'

In [79]:
soup.select_one('div#bodyContent h2:has(span[id$="Levantine_Arabic"]) ~ p:has(.headword)').parent

<div class="mw-parser-output"><div class="disambig-see-also"><i>See also:</i> <b class="Arab"><a href="/wiki/%D8%A3%D8%B3%D8%AA%D9%81%D8%A7%D8%AF" title="أستفاد">أستفاد</a></b>‎</div>
<div aria-labelledby="mw-toc-heading" class="toc" id="toc" role="navigation"><input class="toctogglecheckbox" id="toctogglecheckbox" role="button" style="display:none" type="checkbox"/><div class="toctitle" dir="ltr" lang="en"><h2 id="mw-toc-heading">Contents</h2><span class="toctogglespan"><label class="toctogglelabel" for="toctogglecheckbox"></label></span></div>
<ul>
<li class="toclevel-1 tocsection-1"><a href="#Arabic"><span class="tocnumber">1</span> <span class="toctext">Arabic</span></a>
<ul>
<li class="toclevel-2 tocsection-2"><a href="#Etymology"><span class="tocnumber">1.1</span> <span class="toctext">Etymology</span></a></li>
<li class="toclevel-2 tocsection-3"><a href="#Pronunciation"><span class="tocnumber">1.2</span> <span class="toctext">Pronunciation</span></a></li>
<li class="toclevel-2 t

In [69]:
headers = [span.parent for span in soup.select('div#bodyContent h2 span[id$="Levantine_Arabic"]')]

for header in headers:
    print(header.span.string)
    for el in header.next_siblings:
        if el.name == 'h2':
            break
        if not el.name:
            continue            
        print(el.name, el.get_text())
        print("=" * 20)

South Levantine Arabic
table Rootف ي د‎2 terms
h3 Etymology[edit]
p From Arabic اِسْتَفَادَ‎ (istafāda).

h3 Pronunciation[edit]
ul IPA(key): /is.ta.faːd/, [ɪs.taˈfæːd]
Audio (al-Lidd)(file)
h3 Verb[edit]
p استفاد • (istafād) (form X, present بستفيد‎ (bistafīd))

ol to benefit or profit from, to make use of
‏بصراحة بستفيد كتير من محاضراته.‎‎bi-ṣarāḥa bastafīd ktīr min muḥāḍarātoHonestly, I get a lot out of his lectures.
h4 Conjugation[edit]
table 

    Conjugation of استفاد (istafād)




singular



plural




1st person

2nd person

3rd person

1st person

2nd person

3rd person


past

m

استفدت (istafadt)

استفدت (istafadt)

استفاد (istafād)

استفدنا (istafadna)

استفدتو (istafadtu)

استفادو (istafādu)


f

استفدتي (istafadti)

استفادت (istafādat)


present

m

بستفيد (bastafīd)

بتستفيد (btistafīd)

بستفيد (bistafīd)

منستفيد (mnistafīd)

بتستفيدو (btistafīdu)

بستفيدو (bistafīdu)


f

بتستفيدي (btistafīdi)

بتستفيد (btistafīd)


subjunctive

m

أستفيد (ʔastafīd)

تستفيد (tistafīd)

In [93]:
[a['href'] for a in soup.css.select('div#mw-pages li a')]

['/wiki/9x',
 '/wiki/A',
 '/wiki/a',
 '/wiki/a_c%C3%B2ng',
 '/wiki/A_Di_%C4%90%C3%A0',
 '/wiki/A_Di_%C4%90%C3%A0_Ph%E1%BA%ADt',
 '/wiki/a_dua',
 '/wiki/a_giao',
 '/wiki/a_ha',
 '/wiki/a_ho%C3%A0n',
 '/wiki/A_La_H%C3%A1n',
 '/wiki/A_L%E1%BB%8Bch_S%C6%A1n',
 '/wiki/A_L%E1%BB%8Bch_S%C6%A1n_%C4%90%E1%BA%AFc_L%E1%BB%99',
 '/wiki/a_l%C3%B4',
 '/wiki/a_m%C3%B3c',
 '/wiki/a_phi%E1%BA%BFn',
 '/wiki/A_Ph%C3%BA_H%C3%A3n',
 '/wiki/A_Q',
 '/wiki/a_t%C3%B2ng',
 '/wiki/A_Tu_La',
 '/wiki/%C3%A0',
 '/wiki/%C3%A0_th%E1%BA%BF_%C3%A0',
 '/wiki/%C3%A0_u%C3%B4m',
 '/wiki/%E1%BA%A2',
 '/wiki/%E1%BA%A3',
 '/wiki/%E1%BA%A2_R%E1%BA%ADp',
 '/wiki/%C3%A1',
 '/wiki/%C3%81',
 '/wiki/%C3%81_C%C4%83n_%C4%90%C3%ACnh',
 '/wiki/%C3%81_Ch%C3%A2u',
 '/wiki/%C3%81_ch%C3%A2u',
 '/wiki/%C3%81_%C4%90%C3%B4ng',
 '/wiki/%C3%A1_%C4%91%C3%B9',
 '/wiki/%C3%A1_h%E1%BA%ADu',
 '/wiki/%C3%A1_ho%C3%A1_th%E1%BA%A1ch',
 '/wiki/%C3%A1_h%C3%B3a_th%E1%BA%A1ch',
 '/wiki/%C3%A1_kh%E1%BA%A9u',
 '/wiki/%C3%A1_kh%C3%B4i',
 '/wiki/%C3%A1_kim',
 '